## Procedural memory

In [1]:
%%capture cap --no-stderr
%pip install -U langmem langgraph

In [2]:
from langgraph.store.memory import InMemoryStore

store = InMemoryStore()
store.put(("instructions",), key="agent_instructions", value={"prompt": "Write good emails."})

In [5]:
from langgraph.prebuilt import create_react_agent
from langgraph.config import get_store

def draft_email(to: str, subject: str, body: str):
    """Submit an email draft."""
    return "Draft saved succesfully."

def prompt(state):
    item = store.get(("instructions",), key="agent_instructions")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}"}
    return [sys_prompt] + state['messages']

agent = create_react_agent("gpt-3.5-turbo", prompt=prompt, tools=[draft_email], store=store)

In [6]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content" :"Draft an email to joe@langchain.dev saying that we want to schedule a followup meeting for thursday at noon."}]}
)
result['messages'][1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  draft_email (call_AxcZNDW51cvkLBIqwGxmxTJc)
 Call ID: call_AxcZNDW51cvkLBIqwGxmxTJc
  Args:
    to: joe@langchain.dev
    subject: Follow-up Meeting Schedule
    body: Dear Joe,

I hope this email finds you well. I would like to schedule a follow-up meeting with you on Thursday at noon to discuss our previous conversation further.

Please let me know if this time works for you, and if not, suggest an alternative time that suits your schedule.

Looking forward to our discussion.

Best regards,
[Your Name]


# Update the prompt

In [1]:
from langmem import create_prompt_optimizer

optimizer = create_prompt_optimizer("gpt-3.5-turbo")

AttributeError: module 'openai' has no attribute 'OpenAI'

In [8]:
current_prompt = store.get(("instructions",), key="agent_instructions").value["prompt"]
feedback = {"request": "Always sign off from 'William'; for meeting requests, offer to schedule on Zoom or Google Meet"}

optimizer_result = optimizer.invoke({"prompt": current_prompt, "trajectories": [(result["messages"], feedback)]})

In [9]:
print(optimizer_result)

Write good emails. Remember to always sign off as 'William' and offer to schedule the meeting on Zoom or Google Meet for meeting requests.


In [10]:
store.put(("instructions",), key="agent_instructions", value={"prompt": optimizer_result})

In [11]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content" :"Draft an email to joe@langchain.dev saying that we want to schedule a followup meeting for thursday at noon."}]}
)
result['messages'][1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  draft_email (call_E7ELcSU1VDEWm9KoQu4OmCWR)
 Call ID: call_E7ELcSU1VDEWm9KoQu4OmCWR
  Args:
    to: joe@langchain.dev
    subject: Follow-up Meeting Schedule
    body: Dear Joe,

I hope this email finds you well. I wanted to follow up on our previous discussion and schedule a follow-up meeting for Thursday at noon. This meeting will give us the opportunity to discuss further the points we addressed earlier.

Please let me know if this time works for you, or suggest an alternative time that would be more convenient. I can schedule the meeting on Zoom or Google Meet, whichever platform you prefer.

Looking forward to our meeting and further collaboration.

Best regards,
William


In [12]:
result = agent.invoke(
    {"messages": [
        {"role": "user", "content" : "Let roger@langchain.dev know that the release should be later by 4:00 PM."}]}
)
result['messages'][1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  draft_email (call_syOc49cE4D5BI9geRqfPMUa4)
 Call ID: call_syOc49cE4D5BI9geRqfPMUa4
  Args:
    to: roger@langchain.dev
    subject: Update on Release Timing
    body: Hi Roger,

I hope this message finds you well. I wanted to inform you that the release has been rescheduled to 4:00 PM today. Please adjust your schedule accordingly.

If you have any questions or concerns, feel free to reach out. Thank you for your understanding.

Best regards,
William


## Multi-agent

In [13]:
%%capture cap --no-stderr
%pip install -U langgraph-supervisor

In [19]:
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langgraph.config import get_store

store = InMemoryStore()

store.put(("instructions",), key="email_agent", value={"prompt": "Write good emails. Repeat your draft content to the user after submitting."})
store.put(("instructions",), key="twitter_agent", value={"prompt": "Write fire tweets. Repeat the tweet content to the user upon submission."})

## Email agent
def draft_email(to: str, subject: str, body: str):
    """Submit an email draft."""
    return "Draft saved succesfully."

def prompt_email(state):
    item = store.get(("instructions",), key="email_agent")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}"}
    return [sys_prompt] + state['messages']

email_agent = create_react_agent(
    "gpt-3.5-turbo", 
    prompt=prompt_email, 
    tools=[draft_email], 
    store=store,
    name="email_assistant",
)

## Tweet

def tweet(to: str, subject: str, body: str):
    """Poast a tweet."""
    return "Legendary."

def prompt_social_media(state):
    item = store.get(("instructions",), key="twitter_agent")
    instructions = item.value["prompt"]
    sys_prompt = {"role": "system", "content": f"## Instructions\n\n{instructions}"}
    return [sys_prompt] + state['messages']

social_media_agent = create_react_agent(
    "gpt-3.5-turbo", 
    prompt=prompt_social_media, 
    tools=[tweet], 
    store=store,
    name="social_media_agent",
)

In [20]:
from langgraph_supervisor import create_supervisor

# Create supervisor workflow
workflow = create_supervisor(
    [email_agent, social_media_agent],
    model="gpt-3.5-turbo",
    prompt=(
        "You are a team supervisor managing email and tweet assistants to help with correspondance."
    )
)

# Compile and run
app = workflow.compile(store=store)

In [21]:
result = app.invoke(
    {"messages": [
        {"role": "user", "content" :"Draft an email to joe@langchain.dev saying that we want to schedule a followup meeting for thursday at noon."}]},
)

In [22]:
result["messages"][3].pretty_print()

================================== Ai Message ==================================
Name: email_assistant

I have drafted an email to Joe at joe@langchain.dev regarding scheduling a follow-up meeting for Thursday at noon. Here is the content:

Subject: Follow-Up Meeting Schedule

Hi Joe,

I hope this email finds you well. I would like to schedule a follow-up meeting with you for this Thursday at noon. Please let me know if this timing works for you.

Looking forward to our discussion.

Best regards,
[Your Name]

Please let me know if you would like me to make any changes or if you are ready to send it.


In [23]:
from langmem import create_multi_prompt_optimizer

feedback = {"request": "Always sign off emails from 'William'; for meeting requests, offer to schedule on Zoom or Google Meet"}

optimizer = create_multi_prompt_optimizer("gpt-3.5-turbo")

In [24]:
from langmem import Prompt
?Prompt

Init signature: Prompt(self, /, *args, **kwargs)
Docstring:     
TypedDict for structured prompt management and optimization.

Example:
    ```python
    from langmem import Prompt

    prompt = Prompt(
        name="extract_entities",
        prompt="Extract key entities from the text:",
        update_instructions="Make minimal changes, only address where"
        " errors have occurred after reasoning over why they occur.",
        when_to_update="If there seem to be errors in recall of named entities.",
    )
    ```

The name and prompt fields are required. Optional fields control optimization:
- update_instructions: Guidelines for modifying the prompt
- when_to_update: Dependencies between prompts during optimization

Use in the prompt optimizers.
File:           /opt/anaconda3/envs/llm_agent/lib/python3.13/site-packages/langmem/prompts/types.py
Type:           _TypedDictMeta
Subclasses:     

In [25]:
email_prompt = store.get(("instructions",), key="email_agent").value['prompt']
tweet_prompt = store.get(("instructions",), key="twitter_agent").value['prompt']

email_prompt = {
    "name": "email_prompt",
    "prompt": email_prompt,
    "when_to_update": "Only if feedback is provided indicating email writing performance needs improved."
}
tweet_prompt = {
    "name": "tweet_prompt",
    "prompt": tweet_prompt,
    "when_to_update": "Only if tweet writing generation needs improvement."
}


optimizer_result = optimizer.invoke({"prompts": [tweet_prompt, email_prompt], "trajectories": [(result["messages"], feedback)]})

In [34]:
store.get(("instructions",), key="email_agent")

Item(namespace=['instructions'], key='email_agent', value={'prompt': "Write good emails. Repeat your draft content to the user after submitting. Always sign off as 'William' and offer to schedule meetings on Zoom or Google Meet for meeting requests."}, created_at='2025-03-04T10:35:30.473418+00:00', updated_at='2025-03-04T10:35:30.473421+00:00')

In [26]:
optimizer_result

[{'name': 'tweet_prompt',
  'prompt': 'Write fire tweets. Repeat the tweet content to the user upon submission.',
  'when_to_update': 'Only if tweet writing generation needs improvement.'},
 {'name': 'email_prompt',
  'prompt': "Write good emails. Repeat your draft content to the user after submitting. Always sign off as 'William' and offer to schedule meetings on Zoom or Google Meet for meeting requests.",
  'when_to_update': 'Only if feedback is provided indicating email writing performance needs improved.'}]

In [27]:
store.put(("instructions",), key="email_agent", value={"prompt": optimizer_result[1]['prompt']})

In [28]:
result = app.invoke(
    {"messages": [
        {"role": "user", "content" :"Draft an email to joe@langchain.dev saying that we want to schedule a followup meeting for thursday at noon."}]},
)

In [32]:
result["messages"][1].pretty_print()

================================== Ai Message ==================================
Name: supervisor
Tool Calls:
  transfer_to_email_assistant (call_SMlIX5zaf9Y4uLVDohrl1yhs)
 Call ID: call_SMlIX5zaf9Y4uLVDohrl1yhs
  Args:
    recipient_email: joe@langchain.dev
    subject: Follow-up Meeting Schedule
    message: Hello Joe,

I hope this email finds you well. We would like to schedule a follow-up meeting with you on Thursday at noon. Please let us know if this time works for you.

Looking forward to our discussion.

Best regards,
[Your Name]


{'messages': [HumanMessage(content='Draft an email to joe@langchain.dev saying that we want to schedule a followup meeting for thursday at noon.', additional_kwargs={}, response_metadata={}, id='df8ce96d-71d4-4a74-b7fe-dc41647975ca'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SMlIX5zaf9Y4uLVDohrl1yhs', 'function': {'arguments': '{"recipient_email":"joe@langchain.dev","subject":"Follow-up Meeting Schedule","message":"Hello Joe,\\n\\nI hope this email finds you well. We would like to schedule a follow-up meeting with you on Thursday at noon. Please let us know if this time works for you.\\n\\nLooking forward to our discussion.\\n\\nBest regards,\\n[Your Name]"}', 'name': 'transfer_to_email_assistant'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 92, 'prompt_tokens': 100, 'total_tokens': 192, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_pre